# Park Imminent

## Image processing
Find all images in the media directory, convert them to dithered 1-bit versions.

In [9]:
from PIL import Image

def dither_image(image, max_width, max_height, foreground_color, background_color):
    # Open the image
    img = Image.open(image)
    
    # Resize the image if it is larger than the maximum width or height
    if img.width > max_width or img.height > max_height:
        img.thumbnail((max_width, max_height))
    
    # Convert the image to grayscale
    img = img.convert('L')
    
    # Dither the image using Floyd-Steinberg dithering algorithm
    img = img.convert('1', dither=Image.FLOYDSTEINBERG)
    
    # Create a new image with the specified foreground and background colors
    new_img = Image.new('RGB', img.size, background_color)
    pixels = new_img.load()
    
    # Iterate over each pixel in the dithered image
    for x in range(img.width):
        for y in range(img.height):
            # Get the pixel value (0 or 255)
            pixel = img.getpixel((x, y))
            
            # Set the corresponding pixel in the new image to the foreground or background color
            if pixel == 0:
                pixels[x, y] = foreground_color
            else:
                pixels[x, y] = background_color
    
    return new_img

## Build site

In [10]:
import os
import markdown2
import re
from PIL import ImageColor

# foreground and background colors
default_fg = (102, 102, 204)
default_bg = (255, 255, 249)

# load header, footer
with open('src/templates/header.txt') as f:
    header = f.readlines()

with open('src/templates/footer.txt') as f:
    footer = f.readlines()

# list of tuples with page title, category, slug and date modified
page_list = []

# process pages, build index
for file in os.listdir("src/pages"):
    if file.endswith(".md"):
        print(f'file: {file}')
        with open(f'src/pages/{file}') as f:
            html = markdown2.markdown(f.read(), extras=['fenced-code-blocks', 'latex', 'header-ids', 'smarty-pants', 'toc', 'metadata', 'strike'])

            # set parameters
            print(html.metadata)
            fg = ImageColor.getcolor(html.metadata["fg"], "RGB") if "fg" in html.metadata else default_fg
            bg = ImageColor.getcolor(html.metadata["bg"], "RGB") if "bg" in html.metadata else default_bg

            # find all images in the html
            images = re.findall(r'<img src="(.+?)"', html)
            print(images)
            for image in images:
                if image.startswith("http"):
                    continue
                # otherwise trim the path to the image and remove the extension
                image = os.path.splitext(os.path.basename(image))[0]
                
                # find an image with matching name and any extension
                for img_file in os.listdir("src/media"):
                    # if filename without extension matches the image name
                    if os.path.splitext(img_file)[0] == image:
                        # convert the image to a dithered png
                        # if fg is darker than bg, swap them
                        if sum(fg) > sum(bg):
                            fg, bg = bg, fg
                        dither_image(f'src/media/{img_file}', 800, 800, fg, bg).save(f'site/media/{image}.png')

            # write the html to a file
            print(f'writing {file} to html')
            with open(f'site/pages/{os.path.splitext(file)[0]}.html', 'w') as out:
                out.writelines(header)
                
                if "fg" in html.metadata:
                    # write style element with css to declare --fgcolor variable
                    out.write(f'<style>:root{{--fgcolor: {html.metadata["fg"]}}}</style>')

                if "bg" in html.metadata:
                    out.write(f'<style>:root{{--bgcolor: {html.metadata["bg"]}}}</style>')

                out.write(f'<h1>{html.metadata.get("title", "Untitled")}</h1>')

                # get count of how many <li> elements are in the toc
                if html.toc_html:
                    toc_count = html.toc_html.count('<li>')
                    if toc_count > 4:
                        out.write('<div class="toc"><span><b>On this page</b></span>')
                        out.write(html.toc_html)
                        out.write('</div>')

                out.write(html)
                out.writelines(footer)
                
                # add page title, slug and date modified to page_list
                filename = f'{os.path.splitext(file)[0]}.html'
                page_list.append((html.metadata.get("title", "Untitled"), html.metadata.get("category", "Uncategorised"),  filename, os.path.getmtime(f'src/pages/{file}')))

# now build an index page

# first split the pages into categories
categories = {}
for title, category, filename, date_modified in page_list:
    if category not in categories:
        categories[category] = []
    categories[category].append((title, filename, date_modified))

# sort the tuples in each category by most recent date modified
for category in categories:
    categories[category].sort(key=lambda x: x[2], reverse=True)

# write the index page
with open('site/pages/index.html', 'w') as f:
    index_header = '''
        <!doctype html>
        <html lang="en">
        <head>
            <meta charset="utf-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <link rel="stylesheet" href="../css/style.css">
            <title>Park Imminent</title>
        </head>
        <body>
            <span>Park Imminent</span>
    '''
    f.write(index_header)
    for category in categories:
        f.write(f'<h2>{category}</h2>')
        f.write('<ul class="nobullets">')
        for title, filename, _ in categories[category]:
            f.write(f'<li><a href="{filename}">{title}</a></li>')
        f.write('</ul>')
    f.write('</body></html>')


file: kumashinshi.md
{'title': 'くましんし (Mr. Bear)', 'category': 'Japanese', 'This is an excerpt from a book called『車のいろは空のいろ—白いぼうし』or _The Sky-Coloured Car': 'White Hat_. I translated it for reading practice. The book is aimed at children and written mostly with hiragana, which made it difficult.'}
[]
writing kumashinshi.md to html
file: easy-bread.md
{'title': 'Easy bread', 'category': 'Recipes'}
[]
writing easy-bread.md to html
file: music.md
{'title': 'Music', 'category': 'Art'}
[]
writing music.md to html
file: bread.md
{'title': 'Bread', 'fg': '#0f0', 'bg': '#f0f', 'category': 'Recipes'}
[]
writing bread.md to html
file: minecraft.md
{'title': 'Minecraft', 'category': 'Games'}
[]
writing minecraft.md to html
file: roast-potatoes.md
{'title': 'Roast potatoes', 'category': 'Recipes'}
[]
writing roast-potatoes.md to html
file: chilli-oil.md
{'title': 'Chilli oil', 'category': 'Recipes', "This recipe is from [Omnivore's Cookbook](https": '//omnivorescookbook.com/how-to-make-chili-oil).

# todo
- [ ] handle images that don't look good black/white? maybe fade.
- [ ] build index
- [ ] improve link appearance
- [ ] fix paragraphs - should they just have spaces?
- [ ] allow for non-colour converting images (special filename prefix)
- [ ] 